## **Model**

In [116]:
import numpy as np

def node_score_error(prob):
    '''
        TODO:
        Calculate the node score using the train error of the subdataset and return it.
        For a dataset with two classes, C(p) = min{p, 1-p}
    '''
    return min(prob, 1.0 - prob)

def node_score_entropy(prob):
    '''
        TODO:
        Calculate the node score using the entropy of the subdataset and return it.
        For a dataset with 2 classes, C(p) = -p * log(p) - (1-p) * log(1-p)
        For the purposes of this calculation, assume 0*log0 = 0.
        HINT: remember to consider the range of values that p can take!
    '''
    # HINT: If p < 0 or p > 1 then entropy = 0

    if prob <= 0.0 or prob >= 1.0:
        return 0.0
    
    return -prob * np.log(prob) - (1.0 - prob) * np.log(1.0 - prob)


def node_score_gini(prob):
    '''
        TODO:
        Calculate the node score using the gini index of the subdataset and return it.
        For dataset with 2 classes, C(p) = 2 * p * (1-p)
    '''

    return 2.0 * prob * (1.0 - prob)

class Node:
    '''
    Helper to construct the tree structure.
    '''
    def __init__(self, left=None, right=None, depth=0, index_split_on=0, isleaf=False, label=1):
        self.left = left
        self.right = right
        self.depth = depth
        self.index_split_on = index_split_on
        self.isleaf = isleaf
        self.label = label
        self.thresold = None
        self.info = {} # used for visualization


    def _set_info(self, gain, num_samples):
        '''
        Helper function to add to info attribute.
        You do not need to modify this. 
        '''

        self.info['gain'] = gain
        self.info['num_samples'] = num_samples


class DecisionTree:

    def __init__(self, data, gain_function=node_score_entropy, max_depth=1, sample_weight=None):
        converted_data = []
        for row in data:
            new_row = row.copy()
            new_row[0] = 1 if row[0] == 1 else 0  # Convert -1 to 0
            converted_data.append(new_row)
       
        # Find majority class (now using 0/1)
        self.majority_class = 1 if sum(row[0] for row in converted_data) > len(converted_data) / 2 else 0
        self.max_depth = max_depth
        self.root = Node(label=self.majority_class)
        self.gain_function = gain_function

        indices = list(range(1, len(data[0])))
        self._split_recurs(self.root, converted_data, indices, sample_weight)

    def predict(self, features):
        '''
        Helper function to predict the label given a row of features.
        You do not need to modify this.
        '''
        internal_pred = self._predict_recurs(self.root, features)
        return 1 if internal_pred == 1 else -1  # Convert 0 to -1 for output

    def accuracy(self, data):
        '''
        Helper function to calculate the accuracy on the given data.
        You do not need to modify this.
        '''
        return 1 - self.loss(data)


    def loss(self, data):
        '''
        Helper function to calculate the loss on the given data.
        You do not need to modify this.
        '''
        cnt = 0.0
        test_Y = [row[0] for row in data]
        for i in range(len(data)):
            prediction = self.predict(data[i])
            if (prediction != test_Y[i]):
                cnt += 1.0
        return cnt/len(data)


    def _predict_recurs(self, node, row):
        '''
        Modified to use thresholds for splitting
        '''
        if node.isleaf or node.index_split_on == 0:
            return node.label

        if row[node.index_split_on] <= node.threshold:
            return self._predict_recurs(node.left, row)
        else:
            return self._predict_recurs(node.right, row)
                    
        
    def _is_terminal(self, node, data, indices, sample_weight=None):
        '''
        TODO:
        Helper function to determine whether the node should stop splitting.
        Stop the recursion if:
            1. The dataset (as passed to parameter data) is empty.
            2. There are no more indices to split on.
            3. All the instances in this dataset belong to the same class
            4. The depth of the node reaches the maximum depth.
        Set the node label to be the majority label of the training dataset if:
            1. The number of class 1 points is equal to the number of class 0 points.
            2. The dataset is empty.
        Return:
            - A boolean, True indicating the current node should be a leaf and 
              False if the node is not a leaf.
            - A label, indicating the label of the leaf (or the label the node would 
              be if we were to terminate at that node). If there is no data left, you
              must return the majority class of the training set.
        '''
        y = [row[0] for row in data]

        # Check Cases if the node should stop splitting
        sumy = np.sum(sample_weight[y == 1]) if sample_weight is not None else sum(row[0] for row in data)

        majority_label = 1 if sumy > 0 else 0
            

        if len(set(y)) == 1:
            return True, y[0]
        if len(data) == 0:
            return True, self.majority_class
        if len(indices) == 0:
            return True, majority_label
        
        
        # TODO: Check cases if the node should be set to the majority label of the training dataset
        if node.depth >= self.max_depth:
            return True, majority_label

        return False, majority_label
        

    def _split_recurs(self, node, data, indices, sample_weight=None):
        if sample_weight is None:
            sample_weight = np.ones(len(data)) / len(data)

        node.isleaf, node.label = self._is_terminal(node, data, indices, sample_weight)

        if not node.isleaf:
            max_gain = -1
            best_threshold = None

            for split_index in indices:
                feature_values = sorted(set(row[split_index] for row in data))

                for i in range(len(feature_values) - 1):
                    threshold = (feature_values[i] + feature_values[i + 1]) / 2
                    gain = self._calc_gain(data, split_index, self.gain_function, threshold, sample_weight)

                    if gain > max_gain:
                        max_gain = gain
                        node.index_split_on = split_index
                        best_threshold = threshold

            node._set_info(max_gain, len(data))
            node.threshold = best_threshold

            node.left = Node(depth=node.depth + 1)
            node.right = Node(depth=node.depth + 1)
            indices.remove(node.index_split_on)

            leftData = [row for row in data if row[node.index_split_on] <= node.threshold]
            rightData = [row for row in data if row[node.index_split_on] > node.threshold]

            left_weights = sample_weight[[row[node.index_split_on] <= node.threshold for row in data]]
            right_weights = sample_weight[[row[node.index_split_on] > node.threshold for row in data]]

            self._split_recurs(node.left, leftData, indices, left_weights)
            self._split_recurs(node.right, rightData, indices, right_weights)
        else:
            node._set_info(0, len(data))


    def _calc_gain(self, data, split_index, gain_function, threshold, sample_weight):
        '''
        Calculate the gain of the proposed splitting and return it.
        Gain = C(P[y=1]) - P[x_i=True] * C(P[y=1|x_i=True]) - P[x_i=False] * C(P[y=0|x_i=False])
        Here the C(p) is the gain_function. For example, if C(p) = min(p, 1-p), this would be
        considering training error gain. Other alternatives are entropy and gini functions.
        '''
        y = [row[0] for row in data]
        xi = [1 if row[split_index] > threshold else 0 for row in data]

        if len(y) != 0 and len(xi) != 0:
            # Calculate weighted probabilities
            probY = np.sum(sample_weight[y == 1]) / np.sum(sample_weight)
            probX = np.sum(sample_weight[xi == 1]) / np.sum(sample_weight)

            y1x1 = np.sum(sample_weight[(y == 1) & (xi == 1)])
            y0x0 = np.sum(sample_weight[(y == 0) & (xi == 0)])

            prob1 = y1x1 / np.sum(sample_weight)
            prob2 = y0x0 / np.sum(sample_weight)

            if abs(probX - 0) < 1e-10:
                probxi_true = 0
            else:
                probxi_true = probX * gain_function(prob1 / probX)

            if abs(probX - 1.0) < 1e-10:
                probxi_false = 0.0
            else:
                probxi_false = (1.0 - probX) * gain_function(prob2 / (1.0 - probX))

            gain = gain_function(probY) - probxi_true - probxi_false
            
        else:
            gain = 0

        return gain
    

    

In [117]:
class DecisionTree2:
    def __init__(self, data, gain_function=None, max_depth=40):
        converted_data = []
        for row in data:
            new_row = row.copy()
            new_row[0] = 1 if row[0] == 1 else 0  # Convert -1 to 0
            converted_data.append(new_row)

        self.majority_class = 1 if sum(row[0] for row in converted_data) > len(converted_data) / 2 else 0
        self.max_depth = max_depth
        self.root = Node(label=self.majority_class)
        self.gain_function = gain_function

        indices = list(range(1, len(data[0])))
        self._split_recurs(self.root, converted_data, indices)

    def _split_recurs(self, node, data, indices):
        node.isleaf, node.label = self._is_terminal(node, data, indices)

        if not node.isleaf:
            max_gain = -1
            best_threshold = None

            for split_index in indices:
                feature_values = sorted(set(row[split_index] for row in data))

                for i in range(len(feature_values) - 1):
                    threshold = (feature_values[i] + feature_values[i + 1]) / 2
                    gain = self._calc_gain(data, split_index, self.gain_function, threshold)

                    if gain > max_gain:
                        max_gain = gain
                        node.index_split_on = split_index
                        best_threshold = threshold

            node._set_info(max_gain, len(data))
            node.threshold = best_threshold

            node.left = Node(depth=node.depth + 1)
            node.right = Node(depth=node.depth + 1)
            indices.remove(node.index_split_on)

            leftData = [row for row in data if row[node.index_split_on] <= node.threshold]
            rightData = [row for row in data if row[node.index_split_on] > node.threshold]

            self._split_recurs(node.left, leftData, indices)
            self._split_recurs(node.right, rightData, indices)

    def _is_terminal(self, node, data, indices):
        y = [row[0] for row in data]

        if len(set(y)) == 1:
            return True, y[0]
        if len(data) == 0:
            return True, self.majority_class
        if len(indices) == 0:
            return True, max(set(y), key=y.count)

        if node.depth >= self.max_depth:
            return True, max(set(y), key=y.count)

        return False, None

    def _calc_gain(self, data, split_index, gain_function, threshold):
        y = [row[0] for row in data]
        xi = [1 if row[split_index] > threshold else 0 for row in data]

        # Calculate gain based on the chosen gain function
        # This is a placeholder; you should implement the actual gain calculation logic
        gain = gain_function(y, xi) if gain_function else 0
        return gain

    def predict(self, features):
        '''Helper function to predict the label given a row of features.'''
        node = self.root
        while not node.isleaf:
            if features[node.index_split_on] <= node.threshold:
                node = node.left
            else:
                node = node.right
        return node.label

In [118]:
# HW_adaboost.py
from sklearn.tree import DecisionTreeClassifier  # Importing DecisionTreeClassifier
    
class AdaBoostClassifier:
    """
    AdaBoost (Adaptive Boosting) Classifier
    An ensemble learning algorithm that combines multiple weak classifiers to build a strong classifier.
    """

    def __init__(self, n_estimators=10, max_depth=2):
        """
        Initialize the AdaBoost classifier.

        Parameters:
        - n_estimators: Number of weak classifiers to use.
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth  # Store max_depth for DecisionTree
        self.w = []  # Store the weights of the classifiers
        self.models = []  # Store the weak classifiers

    def train(self, X, y):
        """
        Fit the AdaBoost model to the training data.

        Parameters:
        - X: Training data, shape (n_samples, n_features)
        - y: Target labels, shape (n_samples,)
        """
        n_samples, n_features = X.shape
        # Initialize weights uniformly
        D = np.ones(n_samples) / n_samples  

        for t in range(self.n_estimators):
            
            # sklearn
            # Create a weak classifier (decision stump)
            model = DecisionTreeClassifier(max_depth=2)  
            # Fit the model to the training data
            model.fit(X, y, sample_weight=D)  # Add this line to train the model
            
            # Make predictions
            y_pred1 = model.predict(X)  # Update to use the model's predict method
            print("y_pred1: ", y_pred1)
           
            
            
            # self-implemented
            weak_model = DecisionTree(data=np.column_stack((y, X)), max_depth=self.max_depth) 
            y_pred = np.zeros_like(y)  # Initialize y_pred
            for i, (y_i, x_i) in enumerate(zip(y, X)):
                combined_input = np.append(y_i, x_i)
                y_pred_i = weak_model.predict(combined_input)  # Predictions from the model
                y_pred[i] = y_pred_i  # Update y_pred with the prediction
            print("y_pred: ", y_pred)
            
            weak_model2 = DecisionTree2(data=np.column_stack((y, X)), max_depth=self.max_depth) 
            y_pred2 = np.zeros_like(y)  # Initialize y_pred
            for i, (y_i, x_i) in enumerate(zip(y, X)):
                combined_input = np.append(y_i, x_i)
                y_pred_i = weak_model2.predict(combined_input)  # Predictions from the model
                y_pred2[i] = y_pred_i  # Update y_pred with the prediction
            print("y_pred2: ", y_pred2)
            

            # Calculate the weighted error
            error = np.sum(D * (y_pred != y))  # Weighted error

            # Error cannot be exactly 0.5 because it represents the weighted sum of misclassifications.
            # If error is 0.5, it means the model is performing no better than random guessing,
            # This means the model is not contributing to the ensemble learning process,
            # and the weights D will not be updated, leading to no improvement in the model.
            if error == 0.5:
                print("Warning: Error is 0.5, stopping training.")
                break

            # Calculate the weight for the weak classifier
            w_t = 0.5 * np.log((1.0 - error) / (error + 1e-10))  # Avoid division by zero

            # Update weights for the next iteration
            D *= np.exp(-w_t * y * y_pred)  # Update weights based on prediction
            D /= np.sum(D * np.exp(-w_t * y * y_pred))  # Normalize weights


            self.models.append(weak_model)  # Store the model
            self.w.append(w_t)  # Store the w_t

    def predict(self, X):
        """
        Predict the class labels for the input data.

        Parameters:
        - X: Input data, shape (n_samples, n_features)

        Returns:
        - Predicted class labels, shape (n_samples,)
        """
        pred = np.zeros(X.shape[0])  # Initialize predictions
        for w_i, model in zip(self.w, self.models):
            pred += w_i * model.predict(X)  # Weighted sum of predictions
        return np.sign(pred)  # Return the sign of the predictions
    
    def accuracy(self, X, y):
        """
        Calculate the accuracy of the model.

        Parameters:
        - X: Input data, shape (n_samples, n_features)
        - y: True labels, shape (n_samples,)

        Returns:
        - Accuracy as a float.
        """
        predictions = self.predict(X)  # Get predictions
        accuracy = np.mean(predictions == y)  # Calculate accuracy
        return accuracy


In [119]:
if __name__ == "__main__":
     # Create a simple dataset
    X = np.array([[0, 0], [1, 1], [1, 0], [0, 1], [0, 2], [1, 2]])
    y = np.array([-1, -1, 1, 1, -1, 1])  # Labels should be -1 and 1 for AdaBoost

    # Initialize the AdaBoost classifier
    model = AdaBoostClassifier(n_estimators=10, max_depth=2)

    # Train the model
    model.train(X, y)

    # Make predictions
    predictions = model.predict(X)

    # Calculate accuracy
    accuracy = model.accuracy(X, y)

    # Print results
    print("Predictions:", predictions)
    print("Accuracy:", accuracy)


y_pred1:  [-1 -1 -1 -1 -1  1]
y_pred:  [-1 -1 -1 -1 -1 -1]
y_pred2:  [0 1 1 0 0 1]
Predictions: [0. 0. 0. 0. 0. 0.]
Accuracy: 0.0


## **Check Model**

In [120]:
import pytest
import numpy as np

# Sets random seed for testing purposes
np.random.seed(0)

# Creates Test Models
test_model1 = AdaBoostClassifier(n_estimators=10)
test_model2 = AdaBoostClassifier(n_estimators=50)
test_model3 = AdaBoostClassifier(n_estimators=20)

# Creates Test Data
x1 = np.array([[0, 0], [1, 1], [1, 0], [0, 1]])
y1 = np.array([-1, -1, 1, 1])  # Labels should be -1 and 1 for AdaBoost

x2 = np.array([[0, 0], [1, 1], [1, 0], [0, 1], [0, 2], [1, 2]])
y2 = np.array([-1, -1, 1, 1, -1, 1])  # More complex dataset

x3 = np.array([[0, 0], [1, 1], [2, 2], [3, 3], [4, 4], [5, 5]])
y3 = np.array([-1, -1, 1, 1, 1, 1])  # Another dataset

# Test Model Train
def check_train_dtype(model, X, y):
    assert isinstance(model.models, list)
    assert len(model.models) > 0, "Model should have trained at least one weak learner."
    assert len(model.w) == len(model.models), "Weights should match the number of models."

# Train the models
test_model1.train(x1, y1)
check_train_dtype(test_model1, x1, y1)

test_model2.train(x2, y2)
check_train_dtype(test_model2, x2, y2)

test_model3.train(x3, y3)
check_train_dtype(test_model3, x3, y3)

# Test Model Predictions
def check_test_dtype(pred, X_test):
    assert isinstance(pred, np.ndarray)
    assert pred.ndim == 1 and pred.shape == (X_test.shape[0],)

# Make predictions
pred1 = test_model1.predict(x1)
check_test_dtype(pred1, x1)
assert (pred1 == y1).all(), "Predictions should match the expected labels for model 1."

pred2 = test_model2.predict(x2)
check_test_dtype(pred2, x2)
assert (pred2 == y2).all(), "Predictions should match the expected labels for model 2."

pred3 = test_model3.predict(x3)
check_test_dtype(pred3, x3)
assert (pred3 == y3).all(), "Predictions should match the expected labels for model 3."

# Test Model Accuracy
def check_accuracy(model, X, y, expected_accuracy):
    accuracy = model.accuracy(X, y)
    assert accuracy == expected_accuracy, f"Expected accuracy: {expected_accuracy}, but got: {accuracy}"

# Check accuracy
check_accuracy(test_model1, x1, y1, 1.0)  # Expecting 100% accuracy for this simple case
check_accuracy(test_model2, x2, y2, 1.0)  # Expecting 100% accuracy for this dataset
check_accuracy(test_model3, x3, y3, 1.0)  # Expecting 100% accuracy for this dataset

# Additional Tests for Edge Cases
def test_empty_train():
    with pytest.raises(ValueError):
        test_model1.train(np.array([]), np.array([]))

def test_empty_predict():
    with pytest.raises(ValueError):
        test_model1.predict(np.array([]))

def test_accuracy_empty():
    with pytest.raises(ValueError):
        test_model1.accuracy(np.array([]), np.array([]))

# Run additional edge case tests
test_empty_train()
test_empty_predict()
test_accuracy_empty()

# Print a message indicating the tests have completed
print("All tests completed successfully.")

y_pred1:  [-1 -1  1  1]
y_pred:  [-1 -1 -1  1]
y_pred2:  [0 0 0 1]
y_pred1:  [-1 -1  1  1]
y_pred:  [-1 -1 -1  1]
y_pred2:  [0 0 0 1]
y_pred1:  [-1 -1  1  1]
y_pred:  [-1 -1 -1  1]
y_pred2:  [0 0 0 1]
y_pred1:  [-1 -1  1  1]
y_pred:  [-1 -1 -1  1]
y_pred2:  [0 0 0 1]
y_pred1:  [-1 -1  1  1]
y_pred:  [-1 -1 -1  1]
y_pred2:  [0 0 0 1]
y_pred1:  [-1 -1  1  1]
y_pred:  [-1 -1 -1  1]
y_pred2:  [0 0 0 1]
y_pred1:  [-1 -1  1  1]
y_pred:  [-1 -1 -1  1]
y_pred2:  [0 0 0 1]
y_pred1:  [-1 -1  1  1]
y_pred:  [-1 -1 -1  1]
y_pred2:  [0 0 0 1]
y_pred1:  [-1 -1  1  1]
y_pred:  [-1 -1 -1  1]
y_pred2:  [0 0 0 1]
y_pred1:  [-1 -1  1  1]
y_pred:  [-1 -1 -1  1]
y_pred2:  [0 0 0 1]
y_pred1:  [-1 -1 -1 -1 -1  1]
y_pred:  [-1 -1 -1 -1 -1 -1]
y_pred2:  [0 1 1 0 0 1]


AssertionError: Model should have trained at least one weak learner.